In [1]:
import os
from tqdm import tqdm
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision
torch.cuda.is_available()

True

In [2]:
USE_CUDA = torch.cuda.is_available() and True 
device = torch.device('cuda' if USE_CUDA else 'cpu')
device

device(type='cuda')

In [3]:
# path
now_path = os.getcwd()
train_path = now_path + '/input/data/train/images/'

In [4]:
# class label tagging
class_dict = {
    ('wear', 'male', 'lt30'): (0, []),
    ('wear', 'male', 'lt60'): (1, []),
    ('wear', 'male', 'gte60'): (2, []),
    ('wear', 'female', 'lt30'): (3, []),
    ('wear', 'female', 'lt60'): (4, []),
    ('wear', 'female', 'gte60'): (5, []),
    ('incorrect', 'male', 'lt30'): (6, []),
    ('incorrect', 'male', 'lt60'): (7, []),
    ('incorrect', 'male', 'gte60'): (8, []),
    ('incorrect', 'female', 'lt30'): (9, []),
    ('incorrect', 'female', 'lt60'): (10, []),
    ('incorrect', 'female', 'gte60'): (11, []),
    ('nowear', 'male', 'lt30'): (12, []),
    ('nowear', 'male', 'lt60'): (13, []),
    ('nowear', 'male', 'gte60'): (14, []),
    ('nowear', 'female', 'lt30'): (15, []),
    ('nowear', 'female', 'lt60'): (16, []),
    ('nowear', 'female', 'gte60'): (17, []),
}

all_files = [dir for dir in os.listdir(train_path) if dir[0] != '.']

for file_name in tqdm(all_files):
    dir_path = train_path + file_name
    all_images = [image for image in os.listdir(dir_path) if image[0] != '.']
    info = file_name.split('_')

    gender, age = info[1], int(info[3])
    age = 'lt30' if age < 30 else 'lt60' if age < 60 else 'gte60'
    for img_name in all_images:
        img_info = img_name.split('.')

        if img_info[0][:4] == 'mask':         # Wear
            wear = 'wear'
        elif img_info[0] == 'incorrect_mask': # Incorrect
            wear = 'incorrect'
        elif img_info[0] == 'normal':         # Not Wear
            wear = 'nowear'
        
        class_dict[(wear, gender, age)][1].append(dir_path + '/' + img_name)

100%|██████████| 2700/2700 [00:00<00:00, 32882.75it/s]


In [5]:
train_data = []
for label, img in class_dict.values():
    for img_path in img:
        train_data.append([img_path, label])
len(train_data), train_data[:5]

(18900,
 [['/opt/ml/input/data/train/images/001022_male_Asian_21/mask4.jpg', 0],
  ['/opt/ml/input/data/train/images/001022_male_Asian_21/mask2.jpg', 0],
  ['/opt/ml/input/data/train/images/001022_male_Asian_21/mask1.jpg', 0],
  ['/opt/ml/input/data/train/images/001022_male_Asian_21/mask5.jpg', 0],
  ['/opt/ml/input/data/train/images/001022_male_Asian_21/mask3.jpg', 0]])

In [6]:
class MyDataset(Dataset):
    def __init__(self, data, train=True):
        self.X = [torch.tensor(np.array(Image.open(x))) for x, _ in data]
        self.Y = [torch.tensor(y, dtype=torch.long) for _, y in data]
    
    def __len__(self):
        return len(self.Y)
    
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

In [7]:
dataset = MyDataset(train_data)

In [8]:
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

## Model_train

In [9]:
# hyperparameter
class_num = 18
epochs = 3
learning_rate = 0.001

In [10]:
imagenet_resnet18 = torchvision.models.resnet18(pretrained=True).to(device)
print(imagenet_resnet18.fc.weight.shape)
imagenet_resnet18.fc = torch.nn.Linear(in_features=512, out_features=class_num, bias=True).to(device)
print(imagenet_resnet18.fc.weight.shape)

torch.Size([1000, 512])
torch.Size([18, 512])


In [11]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(imagenet_resnet18.parameters(), lr=learning_rate)

In [12]:
# train
for e in range(epochs):
    train_acc = 0.0
    
    print('EPOCH : ', e+1)
    for batch_id, (x, y) in enumerate(dataloader):
        
        imagenet_resnet18.train()
        
        # Input type (torch.cuda.ByteTensor) and weight type (torch.cuda.FloatTensor) should be the same
        x = x.type(torch.cuda.FloatTensor).to(device)
        y = y.to(device)

        # Given groups=1, weight of size [64, 3, 7, 7], expected input[10, 512, 384, 3] to have 3 channels, but got 512 channels instead
        x = x.permute(0, 3, 1, 2)
        
        optimizer.zero_grad()
        
        logits = imagenet_resnet18(x)
        max_val, preds = torch.max(logits, 1)
        loss = loss_fn(logits, y)

        loss.backward()
        optimizer.step()
        
        acc = (preds == y).sum().data.cpu().numpy() / preds.size()[0]
        
        train_acc += acc
        
        if batch_id % 50 == 0:
            print('Train batch {} loss {:.3f} train {:.3f}'.format(batch_id + 1, loss.data.cpu().numpy(), train_acc / (batch_id + 1)))

EPOCH :  1
Train batch 1 loss 3.119 train 0.078
Train batch 51 loss 0.646 train 0.762
Train batch 101 loss 0.449 train 0.811
Train batch 151 loss 0.189 train 0.830
Train batch 201 loss 0.287 train 0.846
Train batch 251 loss 0.352 train 0.855
EPOCH :  2
Train batch 1 loss 0.259 train 0.922
Train batch 51 loss 0.165 train 0.919
Train batch 101 loss 0.139 train 0.925
Train batch 151 loss 0.246 train 0.928
Train batch 201 loss 0.145 train 0.931
Train batch 251 loss 0.095 train 0.933
EPOCH :  3
Train batch 1 loss 0.147 train 0.953
Train batch 51 loss 0.123 train 0.940
Train batch 101 loss 0.089 train 0.949
Train batch 151 loss 0.084 train 0.955
Train batch 201 loss 0.096 train 0.959
Train batch 251 loss 0.075 train 0.961


In [13]:
## Test

In [14]:
# path
now_path = os.getcwd()
test_path = now_path + '/input/data/eval/images/'

In [24]:
test_data = [[test_path + img_name, -1] for img_name in os.listdir(test_path) if img_name[0] != '.']

In [27]:
test_dataset = MyDataset(test_data)

In [28]:
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

In [32]:
test_pred = []

imagenet_resnet18.eval()

for batch_id, (x, y) in tqdm(enumerate(test_dataloader)):
    
    x = x.type(torch.cuda.FloatTensor).to(device)
    y = y.to(device)
    
    x = x.permute(0, 3, 1, 2)
    
    logits = imagenet_resnet18(x)
    max_val, preds = torch.max(logits, 1)
    
    test_pred.extend(preds.cpu().tolist())   

197it [00:30,  6.44it/s]


In [33]:
len(test_data), len(test_pred)

(12600, 12600)

In [37]:
info = pd.read_csv(now_path + '/input/data/eval/info.csv')

In [40]:
info.ans = test_pred
info.head()

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,7
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,6
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,5
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,1
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0


In [41]:
info.to_csv(now_path + '/submit/submit01.csv', index = False)

In [42]:
pd.read_csv(now_path + '/submit/submit01.csv')

,ImageID,ans
0,cbc5c6e168e63498590db46022617123f1fe1268.jpg,7
1,0e72482bf56b3581c081f7da2a6180b8792c7089.jpg,6
2,b549040c49190cedc41327748aeb197c1670f14d.jpg,5
3,4f9cb2a045c6d5b9e50ad3459ea7b791eb6e18bc.jpg,1
4,248428d9a4a5b6229a7081c32851b90cb8d38d0c.jpg,0
...,...,...
12595,d71d4570505d6af8f777690e63edfa8d85ea4476.jpg,2
12596,6cf1300e8e218716728d5820c0bab553306c2cfd.jpg,0
12597,8140edbba31c3a824e817e6d5fb95343199e2387.jpg,6
12598,030d439efe6fb5a7bafda45a393fc19f2bf57f54.jpg,15
